In [12]:
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F

## Desing 1

In [3]:


X = numpy.random.uniform(-10, 10, 70).reshape(1, 7, -1)
# Y = np.random.randint(0, 9, 10).reshape(1, 1, -1)

class WrenchNet(torch.nn.Module):
    def __init__(self):
        super(WrenchNet, self).__init__()
        self.layer1 = torch.nn.Conv1d(in_channels=6, out_channels=20, kernel_size=3, stride=2)
        self.act1 = torch.nn.ReLU()
        self.bn1 = nn.BatchNorm1d(20)
        self.layer2 = torch.nn.Conv1d(in_channels=20, out_channels=30, kernel_size=3, stride=2)
        self.act2 = torch.nn.ReLU()
        self.bn2 = nn.BatchNorm1d(30)
        self.layer3 = torch.nn.Conv1d(in_channels=30, out_channels=32, kernel_size=2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.act1(x)
        x = self.bn1(x)
        x = self.layer2(x)
        x = self.act2(x)
        x = self.bn2(x)
        x = self.layer3(x)
#         log_probs = torch.nn.functional.log_softmax(x, dim=1)

        return x

model = WrenchNet().double()
# print(model(torch.tensor(X)).size)
X=torch.rand(1,6,18) #Batch x channel x samples
print(X.shape)
print(model(X.double()).shape)

torch.Size([1, 6, 18])
torch.Size([1, 32, 2])


## Desing 2

In [29]:
# Ref : https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf
'''
Input shape : B x 6 x 18
Output shape: B x 16 x 2
'''
class WrenchNet(nn.Module):
    def __init__(self):
        super(WrenchNet, self).__init__()
        self.layer1 = nn.Conv1d(in_channels=6, out_channels=100, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Conv1d(in_channels=100, out_channels=100, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.maxpool = nn.MaxPool1d(3)
        self.layer3 = nn.Conv1d(in_channels=100, out_channels=160, kernel_size=2)
        self.relu3 = nn.ReLU()
        self.layer4 = nn.Conv1d(in_channels=160, out_channels=160, kernel_size=2)
        self.relu4 = nn.ReLU()
        self.globalpool = nn.AvgPool1d(3)
        self.dropout = nn.Dropout()
        self.fc1 = nn.Sequential(nn.Linear(160, 32, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Linear(32,1)

    def forward(self, x):
        x = self.layer1(x)
#         print(x.shape)
        x = self.relu1(x)
        x = self.layer2(x)
#         print(x.shape)
        x = self.relu2(x)
        x = self.maxpool(x)
#         print(x.shape)        
        x = self.layer3(x)
#         print(x.shape)
        x = self.relu3(x)
        x = self.layer4(x)
#         print(x.shape)
        x = self.relu4(x)
        x = self.globalpool(x)
#         print(x.shape)
        
        x = x.view(x.size(0), -1)
#         print(x.shape)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = WrenchNet()
# print(model(torch.tensor(X)).size)
X=torch.rand(1,6,18) #Batch x channel x samples
print(X.shape)
print(model(X.float()))

torch.Size([1, 6, 18])
torch.Size([1, 100, 16])
torch.Size([1, 100, 14])
torch.Size([1, 100, 4])
torch.Size([1, 160, 3])
torch.Size([1, 160, 2])
torch.Size([1, 160, 1])
torch.Size([1, 160])
tensor([[-0.1359]], grad_fn=<AddmmBackward>)


## Desing 3

In [52]:
'''
Input shape : B x 6 x 18
Output shape: B x 16 x 2
'''
class WrenchNet(nn.Module):
    def __init__(self):
        super(WrenchNet, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(108, 256, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc2 = nn.Sequential(nn.Linear(256, 1024, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc3 = nn.Sequential(nn.Linear(1024, 512, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc4 = nn.Sequential(nn.Linear(512, 128, bias=True), nn.ReLU(inplace=True), nn.Dropout(p = 0.5))
        self.fc5 = nn.Linear(128,1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        return x

model = WrenchNet()
# print(model(torch.tensor(X)).size)
X=torch.rand(1,6,18) #Batch x channel x samples
print(X.shape)
print(model(X.float()))

torch.Size([1, 6, 18])
tensor([[-0.0403]], grad_fn=<AddmmBackward>)


# Sample to create NN model

In [15]:
class DQN_Conv1d(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN_Conv1d, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_shape[0], 32, kernel_size=4, stride=4),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Linear(64, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        return self.conv(x)